In [2]:
import os
import pandas as pd
from pathlib import Path
from function import load_data
from nav_research import NavResearch

Welcome to use Wind Quant API for Python (WindPy)!

COPYRIGHT (C) 2024 WIND INFORMATION CO., LTD. ALL RIGHTS RESERVED.
IN NO CIRCUMSTANCE SHALL WIND BE RESPONSIBLE FOR ANY DAMAGES OR LOSSES CAUSED BY USING WIND QUANT API FOR Python.


In [3]:
nav_dfs = pd.read_csv("nav_dfs.csv")

In [4]:
fund_info = load_data("产品代码.xlsx")
fund_info["基金代码"] = fund_info["基金代码"].astype(str)

In [10]:
nav_df = nav_dfs[nav_dfs["基金代码"] == 953832]

In [11]:
nav_df

,基金代码,基金名称,日期,单位净值,累计净值


In [ ]:
nav_df = nav_dfs[nav_dfs["基金代码"] == 9538322]
start_date = (fund_info[fund_info["基金代码"] == fundcode]["成立日期"].iloc[0].strftime("%Y-%m-%d"))
end_date = nav_df["日期"].max().strftime("%Y-%m-%d")
fund_name = nav_df["基金名称"].iloc[0]
file_path = os.path.join("nav_dfs", f"{fundcode}_{fund_name}_{start_date}_{end_date}.csv")
filtered_df = nav_df[nav_df["日期"] >= start_date]
filtered_df.to_csv(file_path, index=False, encoding="utf-8-sig")

In [7]:
import datetime
from pathlib import Path

In [1]:
import pandas as pd
from WindPy import w
w.start()

Welcome to use Wind Quant API for Python (WindPy)!

COPYRIGHT (C) 2024 WIND INFORMATION CO., LTD. ALL RIGHTS RESERVED.
IN NO CIRCUMSTANCE SHALL WIND BE RESPONSIBLE FOR ANY DAMAGES OR LOSSES CAUSED BY USING WIND QUANT API FOR Python.


.ErrorCode=0
.Data=[OK!]

In [2]:
def get_index_rtn(month_startdate,week_startdate,end_date):
    index_code = ["881001.WI","000300.SH", "000905.SH", "000852.SH","8841425.WI","HSTECH.HI","IXIC.GI"]
    index_name = ["万得全A","中证300", "中证500", "中证1000", "万得小市值指数","恒生科技","纳斯达克"]
    error_code, rtn_year = w.wss(index_code, "pct_chg_per",startDate="2024-12-30",endDate=end_date,usedf=True)
    rtn_year.rename(columns={"PCT_CHG_PER": "2025收益"}, inplace=True)
    rtn_year.index = index_name
    error_code, rtn_month = w.wss(index_code, "pct_chg_per",startDate=month_startdate,endDate=end_date,usedf=True)
    rtn_month.rename(columns={"PCT_CHG_PER": "3月收益"}, inplace=True)
    rtn_month.index = index_name
    error_code, rtn_week = w.wss(index_code, "pct_chg_per",startDate=week_startdate,endDate=end_date,usedf=True)
    rtn_week.rename(columns={"PCT_CHG_PER": "近一周收益"}, inplace=True)
    rtn_week.index = index_name
    rtn = pd.concat([rtn_year, rtn_month, rtn_week], axis=1)
    rtn.reset_index(inplace=True)
    rtn.rename(columns={"index": "指数名称"}, inplace=True)
    # 保存到 Excel
    with pd.ExcelWriter(
        "report_data.xlsx", engine="openpyxl", mode="a", if_sheet_exists="replace"
    ) as writer:
        rtn.to_excel(writer, sheet_name="Sheet3", index=False)
    return rtn

In [ ]:
def get_index_rtn(end_date: str) -> pd.DataFrame:
    '''获取基准指数收益率'''
    index_code = ["881001.WI", "000300.SH", "000905.SH", "000852.SH", "8841425.WI", "HSTECH.HI", "IXIC.GI"]
    index_name = ["万得全A", "沪深300", "中证500", "中证1000", "万得小市值指数", "恒生科技", "纳斯达克"]
    end_dt = datetime.datetime.strptime(end_date, "%Y-%m-%d")
    # 自动计算各时间段起始日期
    periods = [
        {
            'type': 'year',
            'start_date': f"{end_dt.year}-01-01",
            'col_name': f"{end_dt.year}收益"
        },
        {
            'type': 'month',
            'start_date': end_dt.replace(day=1).strftime("%Y-%m-%d"),
            'col_name': f"{end_dt.month}月收益"
        },
        {
            'type': 'week',
            'start_date': (end_dt - datetime.timedelta(days=4)).strftime("%Y-%m-%d"),
            'col_name': "近一周收益"
        }
    ]
    dfs = []
    for period in periods:
        error_code, df = w.wss(index_code,"pct_chg_per",startDate=period['start_date'],endDate=end_date,usedf=True)
        df.rename(columns={'PCT_CHG_PER': period['col_name']}, inplace=True)
        df.index = index_name
        dfs.append(df)
    rtn = pd.concat(dfs, axis=1).reset_index()
    rtn.rename(columns={'index': '指数名称'}, inplace=True)
    with pd.ExcelWriter("report_data.xlsx", engine="openpyxl", mode="a", if_sheet_exists="replace") as writer:
        rtn.to_excel(writer, sheet_name="Sheet3", index=False)
    return rtn

In [12]:
get_index_rtn("2025-03-28")

,指数名称,2025收益,3月收益,近一周收益
0,万得全A,0.373872,0.448407,-1.294897
1,沪深300,-1.654458,0.645686,0.012021
2,中证500,0.285886,0.958393,-0.936026
3,中证1000,1.567772,-0.033446,-2.143271
4,万得小市值指数,6.667549,2.247904,-3.112608
5,恒生科技,21.325566,-1.107551,-2.359936
6,纳斯达克,-12.164265,-8.087586,-2.592548


#### 准备工作

In [2]:
fund_info = load_data("产品代码.xlsx")
fund_info["基金代码"] = fund_info["基金代码"].astype(str)

In [3]:
fund_info["大类策略"].unique()

array(['灵活配置', '主观成长', '主观价值', '主观逆向', '300指增', '500指增', '1000指增',
       '小市值指增', '量化选股', '市场中性', '套利', 'CTA', '多策略', '其他'], dtype=object)

#### 删除文件

In [31]:
def delete_csv_files(directory):
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(".csv"):
                os.remove(os.path.join(root, file))

#### 生成nav_dfs

In [4]:
def get_nav_dfs(fund_info):
    files_list_series = pd.Series(
        [
            i
            for i in Path("./nav_data").rglob("*")
            if i.suffix.lower() in {".csv", ".xlsx", ".xls"}
        ]
    )
    nav_dfs = pd.DataFrame()
    for row in fund_info.itertuples(index=False, name=None):
        nav_df_path = files_list_series[
            files_list_series.apply(lambda x: row[1] in x.stem)
        ]
        nav_df = load_data(nav_df_path.iloc[0])
        nav_df["基金代码"] = row[1]
        nav_df["基金名称"] = row[2]
        nav_df = nav_df[["基金代码", "基金名称", "日期", "单位净值", "累计净值"]]
        nav_dfs = pd.concat([nav_dfs, nav_df], ignore_index=True)
    nav_dfs.to_csv("nav_dfs.csv", index=False, encoding="utf-8-sig")

In [5]:
get_nav_dfs(fund_info)

#### 获取新的nav

In [4]:
data_path = Path("销售产品业绩表现监控表20250324-20250328.xlsx")

In [ ]:
def get_new_nav(data_path, fund_info):
    fundcode_list = fund_info["基金代码"].tolist()
    df1 = pd.read_excel(data_path, sheet_name="私募产品")
    df2 = pd.read_excel(data_path, sheet_name="资管产品（私募）")
    # 处理表df1
    df3 = df1[
        ["产品代码", "产品名称", "最新日期", "最新单位净值（元）", "累计净值（元）"]
    ]
    df3.rename(
        columns={
            "产品代码": "基金代码",
            "产品名称": "基金名称",
            "最新日期": "日期",
            "最新单位净值（元）": "单位净值",
            "累计净值（元）": "累计净值",
        },
        inplace=True,
    )
    df3["日期"] = pd.to_datetime(df3["日期"], format="%Y-%m-%d").dt.strftime("%Y-%m-%d")
    # 处理表df2
    df4 = df2[["产品代码", "产品名称", "最新净值日", "单位净值", "累计净值"]]
    df4.rename(
        columns={"产品代码": "基金代码", "产品名称": "基金名称", "最新净值日": "日期"},
        inplace=True,
    )
    df4["日期"] = pd.to_datetime(df4["日期"], format="%Y%m%d").dt.strftime("%Y-%m-%d")
    df4["基金代码"] = df4["基金代码"].astype(str)
    # 合并处理完后的df3、df4
    df5 = pd.concat([df3, df4], ignore_index=True)
    nav_df = df5[df5["基金代码"].isin(fundcode_list)].reset_index(drop=True)
    # 使用map更新"基金名称"列
    fund_name_map = fund_info.set_index('基金代码')['基金名称'].to_dict()
    nav_df['基金名称'] = nav_df['基金代码'].map(fund_name_map)
    return nav_df

In [7]:
nav_df = get_new_nav(data_path, fund_info)

#### 更新nav_dfs

In [15]:
def update_nav_dfs(nav_df):
    nav_dfs = pd.read_csv("nav_dfs.csv")
    nav_dfs = pd.concat([nav_dfs, nav_df], axis=0, ignore_index=True)
    nav_dfs.drop_duplicates(subset=["基金代码", "日期"], inplace=True)
    nav_dfs.sort_values(by=["基金代码", "日期"], inplace=True)
    nav_dfs.to_csv("nav_dfs.csv", index=False, encoding="utf-8-sig")
    return nav_dfs

In [16]:
nav_dfs = update_nav_dfs(nav_df)

In [17]:
nav_dfs

,基金代码,基金名称,日期,单位净值,累计净值
20175,953832,君享大类资产全天候2号,2024-01-19,1.000,1.000
20176,953832,君享大类资产全天候2号,2024-02-23,1.000,1.000
20177,953832,君享大类资产全天候2号,2024-02-26,1.000,1.000
20178,953832,君享大类资产全天候2号,2024-02-27,1.000,1.000
20179,953832,君享大类资产全天候2号,2024-02-28,1.000,1.000
...,...,...,...,...,...
15671,ZZ676C,宽德量化精选2号C,2025-03-10,1.090,1.395
15672,ZZ676C,宽德量化精选2号C,2025-03-14,1.113,1.418
15673,ZZ676C,宽德量化精选2号C,2025-03-17,1.119,1.424
15674,ZZ676C,宽德量化精选2号C,2025-03-18,1.123,1.428


#### 生成单一净值

In [28]:
def update_nav_df(nav_dfs,fund_info):
    fundcode_list = fund_info["基金代码"].unique().tolist()
    # 确保输出目录存在
    os.makedirs("nav_data", exist_ok=True)
    
    # 检查日期列的类型，如果不是 datetime 类型，则转换为 datetime 类型
    if nav_dfs['日期'].dtype != 'datetime64[ns]':
        nav_dfs['日期'] = pd.to_datetime(nav_dfs['日期'], format='%Y-%m-%d')

    # 遍历 nav_dfs 中 "基金代码" 列的唯一值
    for fundcode in fundcode_list:
        nav_df = nav_dfs[nav_dfs["基金代码"] == fundcode]
        start_date = fund_info[fund_info["基金代码"] == fundcode]["成立日期"].iloc[0].strftime('%Y-%m-%d')
        filtered_df = nav_df[nav_df['日期'] >= start_date]
        end_date = nav_df["日期"].max().strftime('%Y-%m-%d')
        fund_name = nav_df["基金名称"].iloc[0]
        file_path = os.path.join("nav_dfs", f"{fundcode}_{fund_name}_{start_date}_{end_date}.csv")
        filtered_df.to_csv(file_path, index=False, encoding="utf-8-sig")

In [32]:
delete_csv_files("nav_dfs")

In [33]:
update_nav_df(nav_dfs,fund_info)

#### 数据可视化

In [37]:
def multi_fund_comparison(tables, fund_name):
    data1 = tables[0]
    data2 = tables[1].head(1)
    data3 = pd.DataFrame()
    data4 = tables[2]
    years = [2019, 2020, 2021, 2022, 2023, 2024, 2025]
    for year in years:
        data3[f"{year}收益"] = data4.loc[
            data4["分年度业绩"] == year, f"{fund_name}_收益"
        ].values
    data = pd.concat([data1, data2, data3], axis=1)
    data.drop(columns=["基准指数", "整体业绩"], inplace=True)
    return data

def get_report_data(fund_info):
    data = pd.DataFrame()
    files_list_series = pd.Series(
        [i for i in Path("./nav_dfs").rglob("*") if i.suffix.lower() in {".csv", ".xlsx",".xls"}]
    )
    for row in fund_info.itertuples(index=False, name=None):
        nav_df_path = files_list_series[files_list_series.apply(lambda x: row[1] in x.stem)]
        assert len(nav_df_path) == 1, "找到多个文件或者没有文件"
        demo = NavResearch(nav_df_path.item(), row[0], row[2], row[3], row[4], row[5])
        demo.get_data()
        tables = demo.get_analysis_table()
        nav_df = multi_fund_comparison(tables, row[2])
        nav_df["策略类型"] = row[0]
        cols = ['策略类型'] + [col for col in nav_df.columns if col != '策略类型']
        nav_df = nav_df[cols]
        data = pd.concat([data, nav_df], axis=0)
    data.to_excel("report_data.xlsx", index=False)

玄元元丰2号A无法推断频率,自动转为周度
聚鸣匠传11号无法推断频率,自动转为周度
聚鸣章玉价值成长3号A无法推断频率,自动转为周度
健顺云6号C无法推断频率,自动转为周度
翊安投资可转债2号无法推断频率,自动转为周度
弘尚资产弘利2号1期无法推断频率,自动转为周度
勤辰创赢成长6号1期无法推断频率,自动转为周度
勤辰金选森裕弘享1号无法推断频率,自动转为周度
格雷长期价值16号B无法推断频率,自动转为周度
彤源同裕1期1号无法推断频率,自动转为周度
石锋资产笃行7号A期无法推断频率,自动转为周度
景林精选FOF子基金GJ2期无法推断频率,自动转为周度
景林景泰丰收GJ2期无法推断频率,自动转为周度
景林景泰优选GJ2期无法推断频率,自动转为周度
沣京公司精选2期无法推断频率,自动转为周度
睿郡财富11号1期无法推断频率,自动转为周度
高毅任昊致福25期A无法推断频率,自动转为周度
旭鑫价值成长7期无法推断频率,自动转为周度
易同精选2号无法推断频率,自动转为周度
宽远安泰成长1号无法推断频率,自动转为周度
合撰价值精选贰号无法推断频率,自动转为周度
明达精选V号1期无法推断频率,自动转为周度
东方港湾汉韵一期无法推断频率,自动转为周度
静瑞灵动增长安泰1号无法推断频率,自动转为周度
兴聚投资可换股债券1号无法推断频率,自动转为周度
高毅臻选FOF21期A无法推断频率,自动转为周度
衍复新擎300增强一号B无法推断频率,自动转为周度
明汯乐享300指数增强1号A无法推断频率,自动转为周度
衍复新擎A号无法推断频率,自动转为周度
灵均君迎指数增强6号无法推断频率,自动转为周度
赫富乐想500指数增强1号无法推断频率,自动转为周度
宽德中证500指数增强22号一期无法推断频率,自动转为周度
衍复新擎1000增强一号A期无法推断频率,自动转为周度
宽德中证1000指数增强7号B无法推断频率,自动转为周度
幻方量化1000指数专享14号2期A无法推断频率,自动转为周度
乾象中证1000指数增强35号1期无法推断频率,自动转为周度
玄信星选1000指数增强1号1期无法推断频率,自动转为周度
衍复新擎小市值增强一号A期无法推断频率,自动转为周度
宽德小洞天3号B无法推断频率,自动转为周度
天演国睿量化精选6期无法推断频率,自动转为周度
宽德量化精选2号C无法推断频率,自动转为周

In [ ]:
def get_report_data(fund_info):
    data = pd.DataFrame()
    files_list_series = pd.Series(
        [
            i
            for i in Path("./nav_dfs").rglob("*")
            if i.suffix.lower() in {".csv", ".xlsx", ".xls"}
        ]
    )
    for row in fund_info.itertuples(index=False, name=None):
        # 找到对应的文件路径，确保唯一
        nav_df_path_mask = files_list_series.apply(lambda x: row[1] in x.stem)
        nav_df_paths = files_list_series[nav_df_path_mask]
        if len(nav_df_paths) != 1:
            raise ValueError(f"找到 {len(nav_df_paths)} 个文件匹配 {row[1]}，期望找到 1 个")
        nav_df_path = nav_df_paths.iloc[0]
        # 假设 NavResearch 是一个类，初始化并获取数据
        demo = NavResearch(nav_df_path, row[0], row[2], row[3], row[4], row[5])
        df_nav, df_return, df_drawdown = demo.get_data()
        tables = demo.get_analysis_table()
        nav_df = single_fund_table(tables, row[2])
        # 添加策略类型
        nav_df["策略类型"] = row[0]
        # 增加近一周收益列
        if len(df_nav["nav_adjusted"]) >= 2:
            nav_df["近一周收益"] = f"{(df_nav['nav_adjusted'].iloc[-1] / df_nav['nav_adjusted'].iloc[-2] - 1):.2%}"
        else:
            nav_df["近一周收益"] = np.nan
        # 特定基金产品设置为 NaN
        specific_list = ["景林景泰优选GJ2期", "景林精选FOF子基金GJ2期", "景林景泰丰收GJ2期", "千宜乐享精选CTA2号"]
        nav_df.loc[nav_df["基金产品"].isin(specific_list), "近一周收益"] = np.nan
        # 自定义排序列顺序
        cols = ["策略类型"] + [col for col in nav_df.columns if col != "策略类型"]
        nav_df = nav_df[cols]
        # 合并数据，使用 ignore_index=True 避免索引重复问题
        data = pd.concat([data, nav_df], axis=0, ignore_index=True)
    # 处理 2025收益 列为数值类型
    data["2025收益数值"] = data["2025收益"].str.rstrip('%').astype(float) / 100
    # 指定策略类型的自定义顺序
    custom_order = [
        "灵活配置", "主观成长", "主观价值", "主观逆向", "300指增", "500指增", "1000指增", "小市值指增", 
        "量化选股", "市场中性", "套利", "CTA", "多策略", "其他"
    ]
    data["策略类型"] = pd.Categorical(data["策略类型"], categories=custom_order, ordered=True)
    # 排序并删除临时列
    data.sort_values(by=["策略类型", "2025收益数值"], ascending=[True, False], inplace=True)
    data.drop(columns=["2025收益数值"], inplace=True)
    # 重命名列
    data.rename(columns={"净值起始日期": "成立日期", "净值结束日期": "最新净值日期"}, inplace=True)
    # 保存到 Excel
    data.to_excel("report_data.xlsx", index=False)

In [ ]:
# 生成可视化表格report_data.xlsx
def get_report_data(fund_info):
    data = pd.DataFrame()
    files_list_series = pd.Series(
        [
            i
            for i in Path("./nav_dfs").rglob("*")
            if i.suffix.lower() in {".csv", ".xlsx", ".xls"}
        ]
    )
    for row in fund_info.itertuples(index=False, name=None):
        nav_df_path = files_list_series[
            files_list_series.apply(lambda x: row[1] in x.stem)
        ]
        assert len(nav_df_path) == 1, "找到多个文件或者没有文件"
        demo = NavResearch(nav_df_path.item(), row[0], row[2], row[3], row[4], row[5])
        df_nav, df_return, df_drawdown = demo.get_data()
        tables = demo.get_analysis_table()
        nav_df = single_fund_table(tables, row[2])
        nav_df["策略类型"] = row[0]
        # 增加近一周收益列
        nav_df["近一周收益"] = f"{(df_nav["nav_adjusted"].iloc[-1] / df_nav["nav_adjusted"].iloc[-2] - 1):.2%}"
        list = ["景林景泰优选GJ2期","景林精选FOF子基金GJ2期","景林景泰丰收GJ2期","千宜乐享精选CTA2号"]
        nav_df.loc[nav_df["基金产品"].isin(list), "近一周收益"] = np.nan
        # 自定义排序列顺序
        cols = ["策略类型"] + [col for col in nav_df.columns if col != "策略类型"]
        nav_df = nav_df[cols]
        data = pd.concat([data, nav_df], axis=0)
        data["2025收益数值"] = data["2025收益"].str.rstrip('%').astype(float) / 100
        # 指定策略类型的自定义顺序
        custom_order = [
            "灵活配置","主观成长","主观价值","主观逆向","300指增","500指增","1000指增","小市值指增","量化选股","市场中性", "套利", "CTA", "多策略", "其他"
        ]
        data["策略类型"] = pd.Categorical(data["策略类型"], categories=custom_order, ordered=True)
        data.sort_values(by=["策略类型", "2025收益数值"], ascending=[True, False], inplace=True)
        data.drop(columns=["2025收益数值"], inplace=True)
        data.rename(columns={"净值起始日期": "成立日期","净值结束日期": "最新净值日期"}, inplace=True)
    data.to_excel("report_data.xlsx", index=False)